##Kaggle Competition

##Setup

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from scipy import stats
import yaml, time, sys, os, glob

import seaborn as sns
sns.set_style("darkgrid")

from IPython.display import display, Markdown
pd.set_option('display.max_columns', None)  

DATASET = "Kaggle_Competition"
SPLIT_TRAINING = True
DEBUG = False
SEED = 42

COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
else:
    ROOT = "./"

In [82]:
if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(d): os.makedirs(d)
  if not os.path.isdir(ROOT): os.makedirs(ROOT)

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['doc','orig','data','output']: makedirs(d)

##Load Dataset

In [83]:
df_customers = pd.read_csv(f"{ROOT}/orig/customers_train.csv")
print(df_customers.shape)
df_customers.head()

(6577, 15)


,Customer,Churn,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income
0,797197508,No,Silver,2020-06-01,1,2315.0,1565,750.0,0.676,48.0,F,Graduate,Married,2,Less than €30K
1,812854728,No,Silver,2020-01-01,1,7645.0,2076,5569.0,0.272,58.0,F,Second level,Single,5,Less than €30K
2,768000743,No,Silver,2020-01-01,2,6394.0,0,6394.0,0.000,55.0,F,Second level,Single,2,Less than €30K
3,722161439,No,Silver,2020-08-01,3,4663.0,0,4663.0,0.000,41.0,F,Graduate,NaN,2,€30K - €50K
4,759029725,No,Silver,2021-02-01,4,1879.0,1486,393.0,0.791,37.0,F,Second level,Single,1,Less than €30K


In [84]:
df_customers_test = pd.read_csv(f"{ROOT}/orig/customers_test.csv")
print(df_customers_test.shape)
df_customers_test.head()

(3542, 14)


,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income
0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,M,Graduate,Single,3,€50K - €80K
1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,F,Unknown,Divorced,3,€30K - €50K
2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,M,Graduate,Single,3,€100K +
3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,F,Primary level,Single,2,Less than €30K
4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,F,MSc,Unknown,4,Less than €30K


In [85]:
df_accounts = pd.read_csv(f"{ROOT}/orig/accounts_train.csv")
print(df_accounts.shape)
df_accounts.head()

(25012, 2)


,Customer,Account
0,828247016,Investment
1,828247016,Current
2,828247016,Credit Card
3,828247016,On Demand Deposit
4,828247016,Mortgage


In [86]:
df_accounts_test = pd.read_csv(f"{ROOT}/orig/accounts_test.csv")
print(df_accounts_test.shape)
df_accounts_test.head()

(13575, 2)


,Customer,Account
0,802643959,Credit Card
1,802643959,Current
2,802643959,Investment
3,802643959,Loan
4,802643959,Deposit


In [87]:
df_transactions = pd.read_csv(f"{ROOT}/orig/transactions_train.csv")
print(df_transactions.shape)
df_transactions.head()

(514611, 3)


,Customer,Date,Amount
0,828247016,2022-03-01,70.94
1,828247016,2022-02-21,52.16
2,828247016,2022-02-19,67.03
3,828247016,2022-02-19,48.24
4,828247016,2022-05-20,10.97


In [88]:
df_transactions_test = pd.read_csv(f"{ROOT}/orig/transactions_test.csv")
print(df_transactions_test.shape)
df_transactions_test.head()

(227447, 3)


,Customer,Date,Amount
0,802643959,2022-01-03,37.22
1,802643959,2022-01-15,83.26
2,802643959,2022-02-25,9.76
3,802643959,2022-01-26,10.33
4,802643959,2022-01-06,10.88


## Clean

###Missing values

In [89]:
df_customers.isna().sum()

Customer                       0
Churn                         21
Card                           0
Start_Date                     0
Customer_Service_Calls         0
Credit_Limit                   0
Total_Revolving_Balance        0
Average_Open_To_Buy            0
Average_Utilisation_Ratio      0
Age                          265
Gender                         0
Education                      0
Marital_Status               475
Dependents                     0
Income                         0
dtype: int64

In [90]:
df_customers_test.isna().sum()

Customer                     0
Card                         0
Start_Date                   0
Customer_Service_Calls       0
Credit_Limit                 0
Total_Revolving_Balance      0
Average_Open_To_Buy          0
Average_Utilisation_Ratio    0
Age                          0
Gender                       0
Education                    0
Marital_Status               0
Dependents                   0
Income                       0
dtype: int64

In [91]:
def customers(df):
  df_tmp = df.copy()
  df_tmp = df_tmp[df.Churn.isna()]
  problem_count = df_tmp.shape[0]
  problem_columns = set(df_tmp.index)
  problem_customer = set(df_tmp.Customer)
  return problem_count, problem_columns, problem_customer
  
problem_count, problem_columns, problem_customer = customers(df_customers)
print(customers(df_customers))

(21, {3202, 5512, 3467, 4748, 3857, 26, 3998, 4898, 1059, 2595, 945, 3890, 3510, 3386, 3656, 2402, 2290, 246, 1142, 1022, 4223}, {725411460, 825637129, 788850968, 780124829, 749277088, 797924903, 782927539, 743426485, 800988348, 823803709, 813994556, 713977405, 794108738, 726513860, 803750605, 806188111, 806106211, 808405740, 804826733, 818533745, 785588858})


In [92]:
print(df_customers.shape)
df_customers.drop(index=list(problem_columns), inplace=True)
print(df_customers.shape)

(6577, 15)
(6556, 15)


In [93]:
print(df_accounts.shape)
df_accounts.drop(df_accounts[df_accounts.Customer.isin(list(problem_customer))].index, inplace=True)
print(df_accounts.shape)

(25012, 2)
(24937, 2)


In [94]:
print(df_transactions.shape)
df_transactions.drop(df_transactions[df_transactions.Customer.isin(list(problem_customer))].index, inplace=True)
print(df_transactions.shape)

(514611, 3)
(513247, 3)


##Modifying

###Account

In [95]:
df_accounts.Account.value_counts()

Credit Card          6556
Current              5342
On Demand Deposit    2921
Deposit              2851
Investment           2692
Mortgage             2491
Loan                 1220
Joint                 864
Name: Account, dtype: int64

In [96]:
def has_account_types(x, account_types):
    return {account_type: account_type in x.values for account_type in account_types}

account_types = ['Credit Card', 'Current', 'On Demand Deposit', "Deposit", "Investment", "Mortgage", "Loan", "Joint"]

grouped_a = df_accounts.groupby('Customer').apply(lambda x: pd.Series(has_account_types(x['Account'], account_types)))
grouped_a = grouped_a.reset_index()

In [97]:
grouped_a_test = df_accounts_test.groupby('Customer').apply(lambda x: pd.Series(has_account_types(x['Account'], account_types)))
grouped_a_test = grouped_a_test.reset_index()

In [98]:
def rename_columns(grouped):
  grouped = grouped.rename(columns={
    'Credit Card': 'HasCreditCard',
    'Current': 'HasCurrent',
    'On Demand Deposit': 'HasOnDemandDeposit',
    'Deposit': 'HasDeposit',
    'Investment': 'HasInvestment',
    'Mortgage': 'HasMortgage',
    'Loan': 'HasLoan',
    'Joint': 'HasJoint'
    })
  return grouped

In [99]:
grouped_a = rename_columns(grouped_a)
grouped_a_test = rename_columns(grouped_a_test)

In [100]:
print(grouped_a.shape)
grouped_a.head()

(6556, 9)


,Customer,HasCreditCard,HasCurrent,HasOnDemandDeposit,HasDeposit,HasInvestment,HasMortgage,HasLoan,HasJoint
0,708115009,True,False,False,False,False,False,False,False
1,708117651,True,True,False,False,False,False,False,False
2,708163600,True,True,True,False,False,False,False,False
3,708166361,True,True,False,True,False,True,False,False
4,708169780,True,True,False,False,False,False,False,False


In [101]:
print(grouped_a_test.shape)
grouped_a_test.head()

(3542, 9)


,Customer,HasCreditCard,HasCurrent,HasOnDemandDeposit,HasDeposit,HasInvestment,HasMortgage,HasLoan,HasJoint
0,708182832,True,True,False,True,True,False,False,False
1,708240660,True,True,False,True,True,False,True,False
2,708249778,True,True,False,False,True,True,False,False
3,708259785,True,True,False,True,False,False,False,False
4,708264579,True,True,True,True,True,False,False,True


###Transactions

In [102]:
grouped_t = df_transactions.groupby('Customer').agg({
    'Date': 'count',
    'Amount': ['sum', 'mean']
})

#get rid of the label "date" and "amount"
grouped_t.columns = grouped_t.columns.droplevel(level=[0])

grouped_t.columns = [c.replace(" ", "_").replace("'", "").replace("?", "").capitalize() for c in grouped_t.columns]

In [103]:
grouped_t_test = df_transactions_test.groupby('Customer').agg({
    'Date': 'count',
    'Amount': ['sum', 'mean']
})

#get rid of the label "date" and "amount"
grouped_t_test.columns = grouped_t_test.columns.droplevel(level=[0])

grouped_t_test.columns = [c.replace(" ", "_").replace("'", "").replace("?", "").capitalize() for c in grouped_t_test.columns]

In [104]:
print(grouped_t.shape)
grouped_t.head()

(7858, 3)


,Count,Sum,Mean
Customer,,,
708115009,36,1886.99,52.416389
708117651,86,7400.03,86.046860
708163600,104,13299.99,127.884519
708166361,72,4107.99,57.055417
708169780,78,4539.95,58.204487


In [105]:
print(grouped_t_test.shape)
grouped_t_test.head()

(3542, 3)


,Count,Sum,Mean
Customer,,,
708182832,64,4341.01,67.828281
708240660,41,2349.01,57.292927
708249778,60,2120.98,35.349667
708259785,121,15195.00,125.578512
708264579,36,1302.02,36.167222


###Merging the tables

In [106]:
merged_df = pd.merge(df_customers, grouped_a, on='Customer')
merged_df = pd.merge(merged_df, grouped_t, on='Customer')

merged_df.head()

,Customer,Churn,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,HasCreditCard,HasCurrent,HasOnDemandDeposit,HasDeposit,HasInvestment,HasMortgage,HasLoan,HasJoint,Count,Sum,Mean
0,797197508,No,Silver,2020-06-01,1,2315.0,1565,750.0,0.676,48.0,F,Graduate,Married,2,Less than €30K,True,True,False,True,True,True,False,False,90,4827.96,53.644000
1,812854728,No,Silver,2020-01-01,1,7645.0,2076,5569.0,0.272,58.0,F,Second level,Single,5,Less than €30K,True,False,False,False,False,False,False,False,75,7709.97,102.799600
2,768000743,No,Silver,2020-01-01,2,6394.0,0,6394.0,0.000,55.0,F,Second level,Single,2,Less than €30K,True,True,False,False,False,False,False,False,69,3470.06,50.290725
3,722161439,No,Silver,2020-08-01,3,4663.0,0,4663.0,0.000,41.0,F,Graduate,NaN,2,€30K - €50K,True,True,False,True,True,True,True,False,81,3831.01,47.296420
4,759029725,No,Silver,2021-02-01,4,1879.0,1486,393.0,0.791,37.0,F,Second level,Single,1,Less than €30K,True,True,False,True,True,True,False,False,88,5202.04,59.114091


In [107]:
merged_df_test = pd.merge(df_customers_test, grouped_a_test, on='Customer')
merged_df_test = pd.merge(merged_df_test, grouped_t_test, on='Customer')

merged_df_test.head()

,Customer,Card,Start_Date,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,HasCreditCard,HasCurrent,HasOnDemandDeposit,HasDeposit,HasInvestment,HasMortgage,HasLoan,HasJoint,Count,Sum,Mean
0,774663629,Gold,2020-01-01,2,29663.0,1743,27920.0,0.059,51,M,Graduate,Single,3,€50K - €80K,True,True,False,True,False,False,False,False,93,14638.00,157.397849
1,720420396,Silver,2020-11-01,2,2032.0,1195,837.0,0.588,45,F,Unknown,Divorced,3,€30K - €50K,True,False,False,False,False,False,False,False,80,4478.97,55.987125
2,815283379,Silver,2020-01-01,1,17268.0,1197,16071.0,0.069,57,M,Graduate,Single,3,€100K +,True,True,False,False,False,False,False,False,90,4077.01,45.300111
3,764861610,Silver,2020-05-01,3,2264.0,0,2264.0,0.000,38,F,Primary level,Single,2,Less than €30K,True,True,False,True,True,False,True,False,28,1204.02,43.000714
4,742798818,Silver,2020-01-01,1,2192.0,1464,728.0,0.668,45,F,MSc,Unknown,4,Less than €30K,True,False,True,True,False,True,True,True,89,4905.03,55.112697


In [108]:
num_features = [c for c in merged_df.select_dtypes(["int","float"]).columns if c not in 'Churn']
print(num_features)

['Customer', 'Customer_Service_Calls', 'Credit_Limit', 'Total_Revolving_Balance', 'Average_Open_To_Buy', 'Average_Utilisation_Ratio', 'Age', 'Dependents', 'Count', 'Sum', 'Mean']


In [109]:
cat_features = [c for c in merged_df.columns if c not in num_features]
print(cat_features)

['Churn', 'Card', 'Start_Date', 'Gender', 'Education', 'Marital_Status', 'Income', 'HasCreditCard', 'HasCurrent', 'HasOnDemandDeposit', 'HasDeposit', 'HasInvestment', 'HasMortgage', 'HasLoan', 'HasJoint']


In [110]:
merged_df[cat_features] = merged_df[cat_features].astype(str)
for c in cat_features: merged_df[c] = pd.Categorical(merged_df[c])

In [111]:
cat_features.remove('Churn')
merged_df_test[cat_features] = merged_df_test[cat_features].astype(str)
for c in cat_features: merged_df_test[c] = pd.Categorical(merged_df_test[c])

In [112]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6556 entries, 0 to 6555
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Customer                   6556 non-null   int64   
 1   Churn                      6556 non-null   category
 2   Card                       6556 non-null   category
 3   Start_Date                 6556 non-null   category
 4   Customer_Service_Calls     6556 non-null   int64   
 5   Credit_Limit               6556 non-null   float64 
 6   Total_Revolving_Balance    6556 non-null   int64   
 7   Average_Open_To_Buy        6556 non-null   float64 
 8   Average_Utilisation_Ratio  6556 non-null   float64 
 9   Age                        6291 non-null   float64 
 10  Gender                     6556 non-null   category
 11  Education                  6556 non-null   category
 12  Marital_Status             6556 non-null   category
 13  Dependents                 6556 n

In [113]:
merged_df.shape

(6556, 26)

In [114]:
merged_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542 entries, 0 to 3541
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Customer                   3542 non-null   int64   
 1   Card                       3542 non-null   category
 2   Start_Date                 3542 non-null   category
 3   Customer_Service_Calls     3542 non-null   int64   
 4   Credit_Limit               3542 non-null   float64 
 5   Total_Revolving_Balance    3542 non-null   int64   
 6   Average_Open_To_Buy        3542 non-null   float64 
 7   Average_Utilisation_Ratio  3542 non-null   float64 
 8   Age                        3542 non-null   int64   
 9   Gender                     3542 non-null   category
 10  Education                  3542 non-null   category
 11  Marital_Status             3542 non-null   category
 12  Dependents                 3542 non-null   int64   
 13  Income                     3542 n

In [115]:
merged_df_test.shape

(3542, 25)

In [116]:
merged_df.isna().sum()

Customer                       0
Churn                          0
Card                           0
Start_Date                     0
Customer_Service_Calls         0
Credit_Limit                   0
Total_Revolving_Balance        0
Average_Open_To_Buy            0
Average_Utilisation_Ratio      0
Age                          265
Gender                         0
Education                      0
Marital_Status                 0
Dependents                     0
Income                         0
HasCreditCard                  0
HasCurrent                     0
HasOnDemandDeposit             0
HasDeposit                     0
HasInvestment                  0
HasMortgage                    0
HasLoan                        0
HasJoint                       0
Count                          0
Sum                            0
Mean                           0
dtype: int64

In [117]:
merged_df['Age'].replace(np.nan, merged_df['Age'].median(), inplace = True)
merged_df['Age'] = merged_df['Age'].astype(int)

##Save

In [118]:
merged_df.to_pickle(f"{ROOT}/data/df_train_clean.pkl")

In [119]:
merged_df_test.to_pickle(f"{ROOT}/data/df_test_clean.pkl")